In [ ]:
#!pip install python-whois
#!pip install requests
#!pip install beautifulsoup4

In [1]:
#Area de imports

import whois
import requests
import time
from bs4 import BeautifulSoup

In [2]:
#Se recuperan la información de los principales Estadios españoles de la wikipedia

#Url de los estadios españoles en la wikipedia
urlEstadios = "https://es.wikipedia.org/wiki/Anexo:Estadios_de_f%C3%BAtbol_de_Espa%C3%B1a"

#Se realiza el request
pageEstadios = requests.get(urlEstadios)

#Se crea el objeto BeautifulSoup
soupEs = BeautifulSoup(pageEstadios.content)

#Se buscan todos los objetos Table del  y se crea una lista con ellos
tables = soupEs.find_all('table')

#Se recupera el objeto tbody de la primera tabla
tbody = tables[0].find('tbody')

#Se recuperan todas las filas del tbody anterior como una lista
trs = tbody.find_all('tr')

#Se borra el primer elemento ya que se trata de la cabecera
del trs[0]

#Se crean los objetos para guardar los elementos
ciudades = []
#estadios = []
equipos = []

#Bucle que recorre todas las filas de la tabla
for tr in trs:
    
    #Se crea una lista con todas la columnas
    tds = tr.find_all('td')
    
    #Se elimina la primera columna si corresponde con un dígito, ya que hay filan con filas espandidas y esto hace que no
    #todas las filas tengan las mismas columnas, esto elimina el problema de la primera columna
    if tds[0].string.isdigit():
        del tds[0]
    
    #Se recupera la ciudad y se inserta en la lista de cuiudades, se tiene en cuenta el problema de las filas expandidas
    if(len(tds) >= 7):
        ciudades.append(tds[1].find('a').string)
    elif (len(tds) == 6):
        ciudades.append(ciudades[-1])

    #estadios.append(tds[0].find('a').string)

    #Se recuperan los equipos, se tiene en cuenta que algunos equipos pueden venir como un enlace y otros como un texto
    try:
        equipos.append(tds[-2].find('a').string)
    except AttributeError as e:
        equipos.append(tds[-2].string)

#Se carga la segunda tabla
tbody = tables[1].find('tbody')

#Se recuperan todas las filas de la segunda tabla
trs = tbody.find_all('tr')

#Se elimina la cabecera
del trs[0]

i = 0

#Recorre las filas de la segunda tabla es el mismo tratamiento que el anterior bucle
for tr in trs:
    
    #Se recuperan los primeros 90 elementos para evitar un error que daria más adelante, se ha decidido que no merece la
    #pena corregir ese error ya qué los datos restantes recuperados no se necesitan ya que son estadios de categorias inferiores.
    if i == 90:
        break

    tds = tr.find_all('td')
       
    if tds[0].string.isdigit():
        del tds[0]
    
    if(len(tds) >= 6):
        #print(tds[1].find('a').string)
        ciudades.append(tds[1].find('a').string)
    elif (len(tds) == 5):
        #print(ciudades2[-1])
        ciudades.append(ciudades[-1])

    #estadios.append(tds[0].find('a').string)
    equipos.append(tds[-1].find('a').string)
    i = i + 1   

dic = {}
for ciudad, equipo in zip(ciudades, equipos):
    
    #Se formatean manualmente los equipo para que coincidan los nombres con los nombres de los equipos recuperados de bdfutbol
    equipo = equipo.replace("C. F.", "").strip()
    equipo = equipo.replace("F. C.", "").strip()
    equipo = equipo.replace("C. D.", "").strip()
    equipo = equipo.replace("U. D.", "").strip()
    equipo = equipo.replace("FC", "").strip()
    equipo = equipo.replace("RCD", "").strip()
    equipo = equipo.replace("R. C.", "").strip()
    equipo = equipo.replace("R.", "").strip()    
    equipo = equipo.replace("CD", "").strip()  
    equipo = equipo.replace("C. A.", "").strip() 
    equipo = equipo.replace("U.D.", "").strip() 
    equipo = equipo.replace("Club ", "").strip() 
    equipo = equipo.replace("Real ", "").strip() 
    equipo = equipo.replace(" Balompié", "").strip() 
    equipo = equipo.replace("SD ", "").strip() 
    equipo = equipo.replace(" URJC", "").strip()
    equipo = equipo.replace("A. D. ", "").strip()
    equipo = equipo.replace(" CF UDS", "").strip()   
    equipo = equipo.replace(" CF", "").strip()  
    equipo = equipo.replace("UD", "").strip()      
    
    #Se crea un diccionario con el nombre del equipo como clave
    dic[equipo] = ciudad

    
#Se añaden al diccionarios aquellos equipos que con los parseos manuales se quedaron con nombres incorrectos
dic['Real Madrid'] = 'Madrid'
dic['Alavés'] = 'Vitoria'
dic['Real Sociedad'] = 'San Sebastián'


print(dic)

{'Barcelona': 'Barcelona', 'Madrid': 'Madrid', 'Atlético de Madrid': 'Madrid', 'Betis': 'Sevilla', 'En mantenimiento': 'Sevilla', 'Desocupado': 'Barcelona', 'Valencia': 'Valencia', 'Athletic Club': 'Bilbao', 'Sevilla': 'Sevilla', 'Espanyol': 'Cornellá de Llobregat', 'Sociedad de Fútbol': 'San Sebastián', 'Elche': 'Elche', 'Zaragoza': 'Zaragoza', 'Deportivo de La Coruña': 'La Coruña', 'Las Palmas': 'Las Palmas de Gran Canaria', 'Murcia': 'Murcia', 'Oviedo': 'Oviedo', 'Málaga': 'Málaga', 'Sporting de Gijón': 'Gijón', 'Hércules': 'Alicante', 'Celta de Vigo': 'Vigo', 'Valladolid': 'Valladolid', 'Levante': 'Valencia', 'Osasuna': 'Pamplona', 'Villarreal': 'Villarreal', 'Mallorca': 'Palma', 'Tenerife': 'Santa Cruz de Tenerife', 'Racing de Santander': 'Santander', 'Recreativo de Huelva': 'Huelva', 'Córdoba': 'Córdoba', 'Xerez Deportivo': 'Jerez de la Frontera', 'Cádiz': 'Cádiz', 'Deportivo Alavés': 'Vitoria', 'Granada': 'Granada', 'Salamanca': 'Villares de la Reina', 'Albacete': 'Albacete', 'G

In [3]:
#Se crea el objeto linea que correspondera a cada linea del csv resultante
class linea(object):
    
    def __init__(self, equipoAnf, equipoVis, resultadoAnf, resultadoVis, fecha, estadio, arbitro):

        #Nombre del equipo anfitrion
        self.equipoAnf = equipoAnf
        
        #Nombre del equipo visitante
        self.equipoVis = equipoVis 
        
        #Num de goles del equipo anfitrion
        self.resultadoAnf = resultadoAnf 
        
        #Num de goles del equipo visitante
        self.resultadoVis = resultadoVis
        
        #Fecha del encuentro
        self.fecha = fecha
        
        #Nombre del estadio
        self.estadio = estadio
        
        #Nombre del arbitro del encuentro
        self.arbitro = arbitro
        
        #Se usa el diccionario creado anteriormente para recuperar la ciudad
        try:
            self.ciudad = dic[equipoAnf]
        except KeyError:
            self.ciudad = "n/d"
        
        
    @property
    def equipoAnf(self):
        return self._equipoAnf
    
    @equipoAnf.setter
    def equipoAnf(self, value):
        self._equipoAnf = value
    
    @property
    def equipoVis(self):
        return self._equipoVis
    
    @equipoVis.setter
    def equipoVis(self, value):
        self._equipoVis = value
    
    @property
    def resultadoAnf(self):
        return self._resultadoAnf
    
    @resultadoAnf.setter
    def resultadoAnf(self, value):
        self._resultadoAnf = value    

    @property
    def resultadoVis(self):
        return self._resultadoVis

    @resultadoVis.setter
    def resultadoVis(self, value):
        self._resultadoVis = value    

    @property
    def fecha(self):
        return self._fecha

    @fecha.setter
    def fecha(self, value):
        self._fecha = value    
        
    @property
    def estadio(self):
        return self._estadio

    @estadio.setter
    def estadio(self, value):
        self._estadio = value      
        
    @property
    def ciudad(self):
        return self._ciudad

    @ciudad.setter
    def ciudad(self, value):
        self._ciudad = value
        
    @property
    def arbitro(self):
        return self._arbitro

    @arbitro.setter
    def arbitro(self, value):
        self._arbitro = value
    


In [19]:
#Se imprime la informacion de la pagina bdfutbol para conocer al propietario
print(whois.whois('https://www.bdfutbol.com/'))

{
  "domain_name": [
    "BDFUTBOL.COM",
    "bdfutbol.com"
  ],
  "registrar": "1&1 IONOS SE",
  "whois_server": "whois.ionos.com",
  "referral_url": null,
  "updated_date": [
    "2020-01-22 08:30:25",
    "2017-02-12 10:43:58"
  ],
  "creation_date": "2008-01-21 16:08:14",
  "expiration_date": "2021-01-21 16:08:14",
  "name_servers": [
    "NS1026.UI-DNS.BIZ",
    "NS1047.UI-DNS.DE",
    "NS1054.UI-DNS.ORG",
    "NS1060.UI-DNS.COM"
  ],
  "status": [
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientTransferProhibited https://www.icann.org/epp#clientTransferProhibited",
    "autoRenewPeriod https://www.icann.org/epp#autoRenewPeriod"
  ],
  "emails": [
    "abuse@ionos.com",
    "dataprivacyprotected@1und1.de"
  ],
  "dnssec": [
    "unsigned",
    "Unsigned"
  ],
  "name": "REDACTED FOR PRIVACY",
  "org": null,
  "address": "REDACTED FOR PRIVACY",
  "city": "REDACTED FOR PRIVACY",
  "state": "GI",
  "zipcode": "REDACTED FOR PRIVACY",
  "countr

In [10]:
#Se crea la url por partes para poder recorrer las diferentes temporadas
main = 'https://www.bdfutbol.com/es/'
tt    = 't/t'
html = '.html'
guion = '-'

urlMainList = []


#for i in range(0, -1, -1):
#se recorre un rango entre 0 y 20 para confeccionar la lista de Url's de las distintas temporadas
for i in range(20, -1, -1):    
    anio1 = str(2000 + i)
    
    if i < 9:
        anio2 = "0" + str(i + 1)
    else:
        anio2 = str(i + 1)
    

    url = main + tt + anio1 + guion + str(anio2) + html
    
    urlMainList.append(url)

In [11]:
print(urlMainList)
#urlMainList[0] = 'https://www.bdfutbol.com/es/t/t2001-02.html'

['https://www.bdfutbol.com/es/t/t2020-21.html', 'https://www.bdfutbol.com/es/t/t2019-20.html', 'https://www.bdfutbol.com/es/t/t2018-19.html', 'https://www.bdfutbol.com/es/t/t2017-18.html', 'https://www.bdfutbol.com/es/t/t2016-17.html', 'https://www.bdfutbol.com/es/t/t2015-16.html', 'https://www.bdfutbol.com/es/t/t2014-15.html', 'https://www.bdfutbol.com/es/t/t2013-14.html', 'https://www.bdfutbol.com/es/t/t2012-13.html', 'https://www.bdfutbol.com/es/t/t2011-12.html', 'https://www.bdfutbol.com/es/t/t2010-11.html', 'https://www.bdfutbol.com/es/t/t2009-10.html', 'https://www.bdfutbol.com/es/t/t2008-09.html', 'https://www.bdfutbol.com/es/t/t2007-08.html', 'https://www.bdfutbol.com/es/t/t2006-07.html', 'https://www.bdfutbol.com/es/t/t2005-06.html', 'https://www.bdfutbol.com/es/t/t2004-05.html', 'https://www.bdfutbol.com/es/t/t2003-04.html', 'https://www.bdfutbol.com/es/t/t2002-03.html', 'https://www.bdfutbol.com/es/t/t2001-02.html', 'https://www.bdfutbol.com/es/t/t2000-01.html']


In [12]:
list_lineas=[]

#Se recorren todas las URL's para recuperar los datos de todas temporadas desde el año 2000 al 2020
for url in urlMainList:
    
    #Se realiza el request
    page = requests.get(url)
    
    #Se parsea el contenido devuelto por el requuest en un objeto beautifulSoup 
    soup = BeautifulSoup(page.content)
    
    #se busca la etiqueta cuya id sea jornada_classi
    jornadas = soup.find(id="jornada_classi")
    
    #se buscan todas las etiquetas a en el objeto jornadas, devuelve una lista
    links = jornadas.find_all('a')
    
    href = []

    #Se recorre la lista de etiquetas a para coger unicamente la href
    for link in links:
        href.append(link.get('href'))
    
    #Se eliminan duplicados
    href_set = set(href)
    
    #Se remplaza la cadena ../ por la página principal para crear la ruta absoluta, con esto se consigue una lista
    #con las url's a cada partido
    href_set = list(map(lambda x: x.replace("../", main), href_set))

    #Se recorre la lista de url's (partidos)
    #for link in href_set[:1]:
    for link in href_set:
        #Se toma una marca de tiempo para usar un delay con el objetivo de no saturar la pagina
        t0 = time.time()
    
        #Se ejecuta el request
        partido = requests.get(link)
        
        #Se parsea el contenido recuperado en el request en un objeto beautifull soup
        p_soup = BeautifulSoup(partido.content)
    
        # estimación del tiempo de respuesta en segundos
        response_delay = time.time() - t0
        # espera de 5x, con respecto al tiempo de respuesta
        time.sleep(5 * response_delay)
    
        #Se recupera el titulo de la página
        Lista = p_soup.title.string.split("-")
        
        #Se recupera el nombre del equipo anfitrion
        Equipo1 = Lista[0].strip()
        
        #Se recupera el nombre del equipo visitante
        Equipo2 = Lista[1].split("(")[0].strip()
        
        #Se recuperan los goles del equipo anfitrion
        Resul1 = Lista[1].split("(")[1].strip()
        
        #Se recuperan los goles del equipo visitante
        Resul2 = Lista[2].split(")")[0].strip()
        
        #Se recupera la fecha del encuentro
        Fecha = Lista[2].split(")")[1].strip()
        
        #Se recupera el estadio usando una busqueda por div y class
        Estadio = p_soup.find_all("div", class_="font-weight-bold")[2].string.strip()
        
        #Se recupera el arbitro usando una busqueda por div y class
        Arbitro = p_soup.find_all("div", class_= "col-sm-4 d-none d-sm-block text-clar")[0].a.get_text()

        #Se cre un objeto linea con los datos recuperados y se añade a la lista de lineas
        list_lineas.append(linea(Equipo1, Equipo2, Resul1, Resul2, Fecha, Estadio, Arbitro))
print("Done")

Done


In [13]:
import csv
filename = 'Partidos.csv'

try:
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for linea in list_lineas:
            writer.writerow([linea.equipoAnf, linea.equipoVis, linea.resultadoAnf, linea.resultadoVis, linea.fecha, linea.estadio, linea.ciudad, linea.arbitro])
except BaseException as e:
    print('BaseException:', filename)
else:
    print('Data has been loaded successfully !')

Data has been loaded successfully !
